In [1]:
!pip install gradio --quiet
!pip install ultralytics --quiet

import gradio as gr
from ultralytics import YOLO
import cv2
from PIL import Image
import numpy as np


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.4/321.4 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 910.2/910.2 kB 44.5 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [2]:
import gdown
model_url = "https://drive.google.com/uc?id=18ziUupmCZKtv_cBzzWhY1t45Dm27ras9"
model_path = "best.pt"  # Save the file in the current working directory

# Download the metadata file
print("Downloading model...")
gdown.download(model_url, model_path, quiet=False)
print(f"Model downloaded successfully as {model_path}")

Downloading...
From: https://drive.google.com/uc?id=18ziUupmCZKtv_cBzzWhY1t45Dm27ras9
To: /content/best.pt
100%|██████████| 6.23M/6.23M [00:00<00:00, 11.2MB/s]

Model downloaded successfully as best.pt


In [3]:
model = YOLO("./best.pt")  # Path to your saved model


In [4]:
# Function to compute CLAHE parameters dynamically
def compute_clahe_parameters(image):
    """
    Compute CLAHE parameters dynamically based on image characteristics.

    Parameters:
        image (numpy.ndarray): Input image in BGR format.

    Returns:
        dict: Dictionary with computed `clipLimit` and `tileGridSize`.
    """
    # Convert image to LAB color space
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, _, _ = cv2.split(lab)

    # Calculate mean brightness of the L channel
    mean_luminance = np.mean(l)

    # Adjust parameters based on mean luminance
    if mean_luminance < 38:  # Very dark image
        clipLimit = 4.0
        tileGridSize = (8, 8)
    elif mean_luminance < 75:  # Moderately dark image
        clipLimit = 3.0
        tileGridSize = (8, 8)
    else:  # Bright image
        clipLimit = 2.0
        tileGridSize = (16, 16)

    return {"clipLimit": clipLimit, "tileGridSize": tileGridSize}

# Function to apply CLAHE preprocessing
def apply_clahe(image):
    """
    Apply CLAHE preprocessing to an input image.

    Parameters:
        image (numpy.ndarray): Input image in BGR format.

    Returns:
        numpy.ndarray: Preprocessed image with CLAHE applied.
    """
    # Convert to LAB color space
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)

    # Compute CLAHE parameters
    clahe_params = compute_clahe_parameters(image)

    # Apply CLAHE to the L channel
    clahe = cv2.createCLAHE(clipLimit=clahe_params["clipLimit"], tileGridSize=clahe_params["tileGridSize"])
    l_clahe = clahe.apply(l)

    # Merge channels back and convert to BGR
    lab_clahe = cv2.merge((l_clahe, a, b))
    enhanced_image = cv2.cvtColor(lab_clahe, cv2.COLOR_LAB2BGR)

    return enhanced_image

In [5]:
# Inference function
def predict(image):
    """
    Run inference on the uploaded image and return the annotated image.

    Parameters:
        image (PIL.Image): Input image uploaded by the user.

    Returns:
        PIL.Image: Annotated image with bounding boxes.
    """
    # Convert PIL image to OpenCV format (BGR)
    img = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)

    # Apply CLAHE preprocessing
    preprocessed_img = apply_clahe(img)

    # Perform prediction
    results = model.predict(source=preprocessed_img, save=False, conf=0.25)

    # Get the annotated image
    annotated_frame = results[0].plot()  # Retrieve the first processed image

    # Convert back to PIL format for Gradio
    return Image.fromarray(cv2.cvtColor(annotated_frame, cv2.COLOR_BGR2RGB))

In [ ]:
# Gradio Interface
demo = gr.Interface(
    fn=predict,  # Inference function
    inputs=gr.Image(type="pil"),  # Input: PIL image
    outputs=gr.Image(type="pil"),  # Output: Annotated image
    title="YOLOv8 Object Detection Demo with CLAHE Preprocessing",  # Title of the demo
    description="Upload an image, and the model will display detected objects with bounding boxes. CLAHE preprocessing is applied to the image.",  # Description
)

# Launch the demo
demo.launch(debug=True)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://41f218b66d46d7bae7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)



0: 448x640 8 Cars, 3 Peoples, 36.9ms
Speed: 3.2ms preprocess, 36.9ms inference, 770.5ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 1 Car, 5 Peoples, 1 Table, 7.9ms
Speed: 6.8ms preprocess, 7.9ms inference, 1.4ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 8 Cars, 3 Peoples, 38.7ms
Speed: 2.4ms preprocess, 38.7ms inference, 1.4ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 1 Car, 5 Peoples, 1 Table, 6.8ms
Speed: 3.3ms preprocess, 6.8ms inference, 1.3ms postprocess per image at shape (1, 3, 448, 640)
